# 사전 준비

In [ ]:
!pip install transformers

# 데이터 불러오기

In [ ]:
import pandas as pd

interview_data = pd.read_csv("/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/Interview_Data.csv")

In [ ]:
interview_data.head(3)

,Question,Answer
0,특별히 회사를 선택한 이유와 입사 후 포부에 대해 기술해주십시오.,"대학교 1학년 2학기, 컴퓨터 프로그래밍 및 실습수업을 통해 프로그래밍에 관심을 가..."
1,지원직무를 선택한 이유와 해당직무에 본인이 적임자라고 생각하는 이유를 기술해주십시오.,"대학교 2학년 1학기, 프로그래밍언어 수업을 통해 영상처리를 처음 접하게 되었습니다..."
2,본인이 다니고 싶은 회사와 동료들은 어떤 모습인지 기술해주십시오.,제가 다니고 싶은 회사는 개인 역량을 강화하는데 도움을 주는 회사입니다. 회사에 취...


In [ ]:
interview_data.tail(3)

,Question,Answer
117,"개발을 하면서 겪었던 기술적인 문제를 서술하고, 그 과정에서 겪었던 시행착오와 그에...",프로그래머는 `조금 더 편리한 방법은 없을까?` 궁금하지 않다면 더 이상의 발전은 ...
118,입사 후 이루고 싶은 최종목표는 무엇입니까?,"입사 후, 기존에 있던 업무 매뉴얼과 문서들을 바탕으로 업무에 빠르게 적응하겠습니다..."
119,성격의 장점과 단점을 기술해 주시기 바랍니다. 입사 후 장점은 어떻게 활용하고 단점...,함께 완성한 것이 혼자 완성한 것보다 결과 그 이상의 가치를 가집니다. 혼자 완성하...


# 텍스트 전처리

In [ ]:
!pip install soynlp

In [ ]:
# 회사이름 리스트를 정규표현식으로 변경
file_path = '/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/companies_name.txt'

with open(file_path) as f:
    names_list = f.read().splitlines()

for i in range(0, len(names_list)):
    if i == 0:
        names_list[i] = str('(' + names_list[i])
    elif i == (len(names_list)-1):
        names_list[i] = str('|' + names_list[i] + ')')
    else:
        names_list[i] = str('|' + names_list[i])

names_str = ''.join(names_list)

print(names_str)

(\x28주\x29동부|AKIS|CJ올리브네트웍스|DB아이앤씨|GS ITM|IBK시스템|KB데이터시스템|KT|KT DS|LG CNS|LG 유플러스|LS아이티씨|NDS|NH농협은행|SK|SK C&C|교보정보통신|농협정보시스템|동국시스템즈|동양네트웍스|동양시스템즈|두산디지털이노베이션|롯데백화점|롯데정보통신|메가존클라우드|메타넷대우정보|미라콤아이앤씨|베스핀글로벌|뷰웍스|비즈테크파트너스|삼성SDS|삼성전자|섹타나인|신세계아이앤씨|신한|신한데이타시스템|신한DS|신한금융그룹|신한카드|쌍용정보통신|쓱|아시아나IDT|우리에프아이에스|우리은행|유호스트|줌인터넷|카카오엔터프라이즈|케이뱅크|코오롱베니트|태광|티몬|티맥스소프트|티시스|포스코ICT|하나금융티아이|한국데이터베이스진흥원|한진정보통신|한화시스템/ICT|현대HDS|현대IT&E|현대모비스|현대아이파크몰|현대오토에버|현대오토에버|휴머스온)


In [ ]:
import re
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
whitespace_symbol = re.compile(r'[\t\r\n\f\v]+')

In [ ]:
def text_preprocess(x):
    x = pattern.sub(' ', x)                     # 일반적으로 사용하는 특수문자, 영어, 한글제외 공백으로 치환
    x = whitespace_symbol.sub(' ', x)           # space(공백문자)를 제외한 whitespace 문자를 공백으로 치환
    x = re.sub(names_str, 'XX', x)
    x = x.strip()                               # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x

In [ ]:
interview_data['Answer'][16]

"OOOOOO에서 주관하는 '제O기 사회리더 대학생 멘토링' 활동 때 팀워크를 발휘하여 공동의 목표 달성에 이바지한 경험이 있습니다. 저희는 고객과 헤어샵 사이 중개 플랫폼을 만들어보자는 목표를 설정하였습니다. 프로젝트에 고객수요조사, 플랫폼 설계 등 혼자 하기에는 무리가 있어서 팀을 구성하였고 저를 포함 5명이 팀을 이루게 되었습니다.\n\n저는 처음 본 조원들에게 효율적인 업무 분담하는 과정이 큰 난관이었습니다. 조원들 간의 장점을 최대한 끌어낼 방법을 찾기 위해 노력했습니다. 장시간 소통을 통해 문제를 해결하는 것이 우선이었습니다. 작품 회의를 하며 여러 업무를 분배하는 과정에서 개인마다 각자 잘할 수 있는 일이 있다는 사실을 인지할 수 있었습니다. 이를 바탕으로, 디자인 경력이 있는 조원에게는 디자인 업무, 평소에 책 읽기를 좋아하고 글 쓰는 재주가 있는 조원에게는 계획서 작성, 플랫폼 구현경험이 있는 조원에게는 App 구성 등 효율적인 업무분담을 할 수 있었습니다.\n\n1년이라는 장기프로젝트를 진행하면서 가장 힘들었던 점은 시간과 공간의 제약이었습니다. 각자의 전공공부와 프로젝트를 병행하는 시간적인 제약과 각각 다른 학교 출신으로 공간적인 제약을 조율하는 것은 정말 힘든 일이었습니다. 프로젝트를 자율적으로 진행하다 보니 진행 상황이 더뎌지는 상황을 확인할 수 있었습니다. 저희는 주 1회 정기적으로 모여 개개인의 업무 상황을 보고하고 서로의 업무진행에 대해 피드백하기로 했습니다. 피드백을 통해 구체적인 메시지를 전달하여 서로가 학습하고 성장할 수 있게 만들었습니다.\n\n팀워크의 가치를 현대모비스에서 실행해보고 싶습니다. 협동 부서의 역할을 이해하고 소통하여 팀원 간, 부서 간의 징검다리가 되겠습니다."

In [ ]:
interview_data.Question = interview_data.Question.apply(text_preprocess)
interview_data.Answer = interview_data.Answer.apply(text_preprocess)

In [ ]:
interview_data['Answer'][16]

"OO에서 주관하는 '제O기 사회리더 대학생 멘토링' 활동 때 팀워크를 발휘하여 공동의 목표 달성에 이바지한 경험이 있습니다. 저희는 고객과 헤어샵 사이 중개 플랫폼을 만들어보자는 목표를 설정하였습니다. 프로젝트에 고객수요조사, 플랫폼 설계 등 혼자 하기에는 무리가 있어서 팀을 구성하였고 저를 포함 5명이 팀을 이루게 되었습니다. 저는 처음 본 조원들에게 효율적인 업무 분담하는 과정이 큰 난관이었습니다. 조원들 간의 장점을 최대한 끌어낼 방법을 찾기 위해 노력했습니다. 장시간 소통을 통해 문제를 해결하는 것이 우선이었습니다. 작품 회의를 하며 여러 업무를 분배하는 과정에서 개인마다 각자 잘할 수 있는 일이 있다는 사실을 인지할 수 있었습니다. 이를 바탕으로, 디자인 경력이 있는 조원에게는 디자인 업무, 평소에 책 읽기를 좋아하고 글 쓰는 재주가 있는 조원에게는 계획서 작성, 플랫폼 구현경험이 있는 조원에게는 App 구성 등 효율적인 업무분담을 할 수 있었습니다. 1년이라는 장기프로젝트를 진행하면서 가장 힘들었던 점은 시간과 공간의 제약이었습니다. 각자의 전공공부와 프로젝트를 병행하는 시간적인 제약과 각각 다른 학교 출신으로 공간적인 제약을 조율하는 것은 정말 힘든 일이었습니다. 프로젝트를 자율적으로 진행하다 보니 진행 상황이 더뎌지는 상황을 확인할 수 있었습니다. 저희는 주 1회 정기적으로 모여 개개인의 업무 상황을 보고하고 서로의 업무진행에 대해 피드백하기로 했습니다. 피드백을 통해 구체적인 메시지를 전달하여 서로가 학습하고 성장할 수 있게 만들었습니다. 팀워크의 가치를 XX에서 실행해보고 싶습니다. 협동 부서의 역할을 이해하고 소통하여 팀원 간, 부서 간의 징검다리가 되겠습니다."

# 데이터 구축

HuggingFace's BPEtokenizer

In [ ]:
tokenize_data = interview_data.values.tolist()

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="<unk>"))
trainer = BpeTrainer(
                vocab_size = 4000,          # vocab_size의 최댓값은 4375
                limit_alphabet = 10000,
                min_frequency = 3,
                special_tokens = ["<unk>", "<s>", "</s>", "<pad>", "<mask>"],
                continuing_subword_prefix = "▁"
                )

tokenizer.pre_tokenizer = Whitespace()

tokenizer.train_from_iterator(tokenize_data, trainer)

In [ ]:
tokenizer.save('/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/BPE_tokenizer.json')

**데이터셋 구축**

In [ ]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file='/content/drive/Othercomputers/내 컴퓨터/Interview_Writer/BPE_tokenizer.json')

In [ ]:
import numpy as np
from torch.utils.data import Dataset

class DataSet(Dataset):
    def __init__(self, docs, tokenizer, max_len):
        super().__init__()
        self.docs = docs
        self.tokenizer = tokenizer
        self.max_len = max_len

        self.bos_id = tokenizer.convert_tokens_to_ids("<s>")
        self.eos_id = tokenizer.convert_tokens_to_ids("</s>")
        self.pad_id = tokenizer.convert_tokens_to_ids("<pad>")

    # padding 및 데이터 size 일치화 함수 (input_ids)
    def add_padding(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_id]*(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]
            
        return inputs

    def __len__(self):  
        return len(self.docs)

    def __getitem__(self, idx):
        Q = self.docs.Question[idx]
        A = self.docs.Answer[idx]

        # <s> Q </s> A </s> <pad>...
        input_ids = [self.bos_id] + self.tokenizer.encode(Q) + [self.eos_id] + self.tokenizer.encode(A) + [self.eos_id]
        input_ids = self.add_padding(input_ids)

        # A </s> <pad>...
        labels = self.tokenizer.encode(A) + [self.eos_id]
        labels = self.add_padding(labels)

        return {'input_ids': np.array(input_ids, dtype=np.int_),
                'labels': np.array(labels, dtype=np.int_)}

In [ ]:
Q_total = 0
A_total = 0

for i in range(0, len(interview_data)):

    question_length = len(interview_data.Question[i])
    answer_length = len(interview_data.Answer[i])

    Q_total += question_length
    A_total += answer_length

print("질문 평균길이 : ", (Q_total/len(interview_data)))
print("답 평균길이 : ", (A_total/len(interview_data)))

질문 평균길이 :  49.475
답 평균길이 :  651.2166666666667


In [ ]:
train_set = DataSet(interview_data, tokenizer=fast_tokenizer, max_len=1024)

In [ ]:
train_set[0]

{'input_ids': array([   1, 3882, 1182, ...,    3,    3,    3]),
 'labels': array([2096,   18, 1929, ...,    3,    3,    3])}

**데이터로더 구축**

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_set, shuffle=False, batch_size=1)

In [ ]:
next(iter(train_dataloader))

{'input_ids': tensor([[   1, 3882, 1182,  ...,    3,    3,    3]]),
 'labels': tensor([[2096,   18, 1929,  ...,    3,    3,    3]])}

# 모델 정의 

In [ ]:
import torch
import torch.nn as nn

from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions
from typing import Optional, Union, Tuple

In [ ]:
class Model(nn.Module):
    def __init__(self, vocab_size, hidden_size=256, max_position_embeddings=1024):
        super(Model, self).__init__()

        self.embed_dim = hidden_size

        self.wte = nn.Embedding(vocab_size, self.embed_dim)     # vocab_size * hidden_size
        self.wpe = nn.Embedding(max_position_embeddings, self.embed_dim)    # max_position_embeddings * hidden_size

        self.drop = nn.Dropout(p = 0.1)
        self.h = nn.ModuleList()    # BertLayer/GPT2Block과 같은 layer를 n층으로 쌓아야함
        self.ln_f = nn.LayerNorm(self.embed_dim, eps=1e-5)
        self.lm_head = nn.Linear(self.embed_dim, vocab_size, bias=False)

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        labels: Optional[torch.LongTensor] = None
        ) -> Union[Tuple, CausalLMOutputWithCrossAttentions]:
        
        """Embedding Layer"""
        # Word Embedding 입력 = input_ids
        input_shape = input_ids.size()
        input_ids = input_ids.view(-1, input_shape[-1])
        batch_size = input_ids.shape[0]

        # Token Type Embedding 입력 = token_type_ids
        token_type_ids = token_type_ids.view(-1, input_shape[-1])

        # Position Embedding 입력 = position_ids (create)
        past_length = 0     # past_key_values is None
        past_key_values = tuple([None] * len(self.h))
        position_ids = torch.arange(past_length, input_shape[-1] + past_length, dtype=torch.long, device=device)
        position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])

        # 2D attention으로 3D attention을 만듦([batch_size, num_heads, from_seq_length, to_seq_length])
        attention_mask = attention_mask.view(batch_size, -1)
        attention_mask = attention_mask[:, None, None, :]
        attention_mask = attention_mask.to(dtype=self.dtype)    # fp16 compatibility (GPU 16비트 부종소수점 호환성)
        attention_mask = (1.0 - attention_mask) * torch.finfo(self.dtype).min
        
        # Word Embedding
        inputs_embeds = self.wte(input_ids)
        
        # Position Embedding
        position_embeds = self.wpe(position_ids)
        hidden_states = inputs_embeds + position_embeds
        
        # Token Type Embedding
        token_type_embeds = self.wte(token_type_ids)
        hidden_states = hidden_states + token_type_embeds

        # Dropout
        hidden_states = self.drop(hidden_states)

        output_shape = input_shape + (hidden_states.size(-1),)
        """ModuleList (attention, multi-layer perceptron)"""
        """LayerNorm"""
        """Linear layer"""